In [1]:
import sys
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision.io import read_image
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F
from torch.optim import Adam
import tqdm
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F
from torch.optim import Adam
import torcheval.metrics as metrics
import json
import argparse
import numpy as np

In [2]:
args = {
    'modelfile':'wikiart.pth',
    'trainingdir':'/home/guserbto@GU.GU.SE/wikiart/train',
    'testingdir': '/home/guserbto@GU.GU.SE/wikiart/test',
    'device': 'cuda:2',
    'epochs':10,
    'batch_size':32,
    
}

modelfile = args['modelfile']
trainingdir = args['trainingdir']
testingdir = args['testingdir']
device = args['device']
epochs = args['epochs']
batch_size = args['batch_size']

In [3]:
class WikiArtImage:
    def __init__(self, imgdir, label, filename):
        self.imgdir = imgdir
        self.label = label
        self.filename = filename
        self.image = None
        self.loaded = False

    def get(self):
        if not self.loaded:
            self.image = read_image(os.path.join(self.imgdir, self.label, self.filename)).float()
            self.loaded = True

        return self.image

In [4]:
class WikiArtDataset(Dataset):
    def __init__(self, imgdir, device="cpu"):
        walking = os.walk(imgdir)
        filedict = {}
        indices = []
        classes = set()
        print("Gathering files for {}".format(imgdir))
        for item in walking:
            sys.stdout.write('.')
            arttype = os.path.basename(item[0])
            artfiles = item[2]
            for art in artfiles:
                filedict[art] = WikiArtImage(imgdir, arttype, art)
                indices.append(art)
                classes.add(arttype)
        print("...finished")
        self.filedict = filedict
        self.imgdir = imgdir
        self.indices = indices
        self.classes = list(classes)
        self.device = device
        
    def __len__(self):
        return len(self.filedict)

    def __getitem__(self, idx):
        imgname = self.indices[idx]
        imgobj = self.filedict[imgname]
        ilabel = self.classes.index(imgobj.label)
        image = imgobj.get().to(self.device)

        return image, ilabel

In [5]:
class WikiArtModel(nn.Module):
    def __init__(self, num_classes=27):
        super().__init__()

        self.conv2d = nn.Conv2d(3, 32, (4, 4), padding=2)
        self.maxpool2d = nn.MaxPool2d((2, 2), padding=0)
        self.flatten = nn.Flatten()
        self.batchnorm1d = nn.BatchNorm1d(32 * 208 * 208)  # True output shape after flattening
        self.linear1 = nn.Linear(32 * 208 * 208, 300)
        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(300, num_classes)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, image):
        output = self.conv2d(image)
        #print("After conv2d:", output.size())  # (batch_size, 32, H_out, W_out)
        output = self.maxpool2d(output)
        #print("After maxpool2d:", output.size())  # (batch_size, 32, H_out/2, W_out/2)
        output = self.flatten(output)
        #print("After flatten:", output.size())  # (batch_size, 32 * H_out/2 * W_out/2)
        output = self.batchnorm1d(output)
        #print("After batchnorm1d:", output.size())  # (batch_size, 32 * H_out/2 * W_out/2)
        output = self.linear1(output)
        output = self.dropout(output)
        output = self.relu(output)
        output = self.linear2(output)
        return self.softmax(output)

In [6]:
def calculate_weights(dataset, num_classes):
    class_counts = np.zeros(num_classes)
    for idx in range(len(dataset)):
        _, label = dataset[idx]
        class_counts[label] += 1

    weights = 1. / class_counts
    sample_weights = np.zeros(len(dataset))
    for idx in range(len(dataset)):
        _, label = dataset[idx]
        sample_weights[idx] = weights[label]

    return sample_weights

In [7]:
# TRAIN

print("Running...")


#traindataset = WikiArtDataset(trainingdir, device)
#testingdataset = WikiArtDataset(testingdir, device)

#print(traindataset.imgdir)

#the_image, the_label = traindataset[5]
#print(the_image, the_image.size())

print("Running...")

traindataset = WikiArtDataset(trainingdir, device)
num_classes = len(traindataset.classes)

def train(epochs=3, batch_size=32, modelfile=None, device="cpu"):
    # Calculate sample weights and create a sampler
    num_classes = len(traindataset.classes)
    sample_weights = calculate_weights(traindataset, num_classes)
    sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)
    
    # Create DataLoader with the sampler
    trainloader = DataLoader(traindataset, batch_size=batch_size, sampler=sampler)
    
    model = WikiArtModel(num_classes=num_classes).to(device)
    optimizer = Adam(model.parameters(), lr=0.01)
    criterion = nn.NLLLoss().to(device)

    for epoch in range(epochs):
        print(f"Starting epoch {epoch}")
        accumulate_loss = 0
        for batch_id, batch in enumerate(tqdm.tqdm(trainloader)):
            X, y = batch
            y = y.to(device)
            optimizer.zero_grad()
            output = model(X)
            loss = criterion(output, y)
            loss.backward()
            accumulate_loss += loss.item()
            optimizer.step()

        print(f"In epoch {epoch}, loss = {accumulate_loss}")

    if modelfile:
        torch.save(model.state_dict(), modelfile)

    return model

Running...
Running...
Gathering files for /home/guserbto@GU.GU.SE/wikiart/train
...............................finished


In [8]:
model = train(args["epochs"], args["batch_size"], modelfile=args["modelfile"], device=device)

Starting epoch 0


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 418/418 [01:29<00:00,  4.65it/s]


In epoch 0, loss = 357375.4498960972
Starting epoch 1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 418/418 [01:30<00:00,  4.64it/s]


In epoch 1, loss = 1495.6514103412628
Starting epoch 2


...

In epoch 7, loss = 1378.666454076767
Starting epoch 8


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 418/418 [01:31<00:00,  4.57it/s]


In epoch 8, loss = 1378.698842048645
Starting epoch 9


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 418/418 [01:29<00:00,  4.69it/s]


In epoch 9, loss = 1378.917324066162


In [9]:
# TEST

#parser = argparse.ArgumentParser()
#parser.add_argument("-c", "--config", help="configuration file", default="config.json")

#args = parser.parse_args()

#config = json.load(open(args.config))

#testingdir = config["testingdir"]
#device = config["device"]


print("Running...")

#traindataset = WikiArtDataset(trainingdir, device)
testingdataset = WikiArtDataset(testingdir, device)

def test(modelfile=None, device="cpu"):
    loader = DataLoader(testingdataset, batch_size=1)

    model = WikiArtModel()
    model.load_state_dict(torch.load(modelfile, weights_only=True))
    model = model.to(device)
    model.eval()

    predictions = []
    truth = []
    for batch_id, batch in enumerate(tqdm.tqdm(loader)):
        X, y = batch
        y = y.to(device)
        output = model(X)
        predictions.append(torch.argmax(output).unsqueeze(dim=0))
        truth.append(y)

    #print("predictions {}".format(predictions))
    #print("truth {}".format(truth))
    predictions = torch.concat(predictions)
    truth = torch.concat(truth)
    metric = metrics.MulticlassAccuracy()
    metric.update(predictions, truth)
    print("Accuracy: {}".format(metric.compute()))
    confusion = metrics.MulticlassConfusionMatrix(27)
    confusion.update(predictions, truth)
    print("Confusion Matrix\n{}".format(confusion.compute()))

Running...
Gathering files for /home/guserbto@GU.GU.SE/wikiart/test
..............................finished


In [10]:
test(modelfile=modelfile, device=device)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 629/629 [00:03<00:00, 161.01it/s]


Accuracy: 0.014308426529169083
Confusion Matrix
tensor([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  23.,   0.,
           0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   6.,   0.,
           0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  45.,   0.,
           0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   9.,   0.,
           0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  82.,   0.,
           0.,   0.,   0.],
  